In [1]:
import numpy as np
import pandas as pd
import torch,sklearn
from sklearn import model_selection
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torcheval import metrics

In [392]:
def load_data(datastr : str,features,target):
    def month_fun(m):
        d = {1:0,2:0,3:0,4:0,5:0,6:0,7:0,8:0,9:0,10:0,11:0,12:0}
        d[m] += 1.0
        return d
    def stat_fun(s):
        d = {'USW00012839' : 0, 'USW00014819' :0, 'USW00013904' :0, 'USW00094728': 0}
        d[s] += 1.0
        return d

    dataset = pd.read_csv(datastr)
    dataset = dataset[features+[target,"Month","STATION"]]
    dataset.dropna(inplace=True)
    dataset[target] = dataset[target].astype("float")
    for f in features:
        dataset[f] = dataset[f].astype("float")
    dataset[["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"]] = \
        dataset.apply(lambda x : month_fun(x["Month"]),axis = 1,result_type="expand")
    dataset[['USW00012839', 'USW00014819', 'USW00013904', 'USW00094728']] =\
            dataset.apply(lambda x : stat_fun(x["STATION"]),axis = 1, result_type="expand")
    dataset.dropna(inplace=True)
    dataset.drop(columns = ["Month","STATION"],inplace = True)
    train,test = model_selection.train_test_split(dataset,test_size = 0.1,random_state = 3)
    train_x = train.drop(columns = [target])
    train_y = train[target]
    test_x = test.drop(columns = [target])
    test_y = test[target]
    return train_x,train_y,test_x,test_y

class NN(nn.Module):
    def __init__(self,input_size,hidden_size):
        super(NN,self).__init__()
        # define layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size = self.input_size, hidden_size = self.hidden_size,\
        dtype = torch.float32,batch_first=True,nonlinearity = 'relu',dropout = 0.1)
        self.rnn2 = nn.RNN(input_size = self.input_size, hidden_size = self.hidden_size,\
        dtype = torch.float32,batch_first=True)
        self.h_n = torch.zeros(1,64,self.hidden_size).requires_grad_()
    # define forward function
    def forward(self, rnninput,inputs):
        rnninput = rnninput.view(rnninput.shape[0],rnninput.shape[1],1).float()
        inputs = inputs.view(inputs.shape[0],inputs.shape[1]).float()
        x, self.h_ = self.rnn(rnninput,self.h_n.detach())
        x2, _ = self.rnn2(rnninput,self.h_n.detach())
        x = nn.ReLU()(x)
        # print(x[:,-1,:].size())
        # print(inputs.size())
        x = torch.cat((inputs,x[:,-1,:]),1)
        # print(x[0])
        # print(x[:,-1,:][0])
        # print(x[:,-1,:][0])
        x = nn.Linear(23,1,dtype=torch.float32)(x)
        x = nn.ReLU()(x)
        # print(x[:5])
        # x = nn.ReLU()(x)
        # x = nn.Linear(16,8,dtype=torch.float32)(x)
        # x = nn.ReLU()(x)
        # x = nn.Linear(8,1,dtype=torch.float32)(x)
        return x

model = NN(1,7)

In [168]:
data = []
for i in range(1,8):
    data.append("d" + str(i))
train_x,train_y,test_x,test_y = load_data("1948_2024.csv",data,"TMAX")

In [393]:
criterion = nn.MSELoss()
tl0 = np.inf

# Defining optimizer
optimizer = optim.Adam(model.parameters(), lr=0.03)
print(train_x.values)
train_loader = DataLoader(list(zip(torch.tensor(train_x.values),torch.tensor(train_y.values))),batch_size  = 64)
test_loader = DataLoader(list(zip(torch.tensor(test_x.values),torch.tensor(test_y.values))),batch_size  = 64)
import time
epoch = 0
while True:

    # Initialising statistics that we will be tracking across epochs
    start_time = time.time()
    total_correct = 0
    total = 0
    total_loss = 0

    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if len(inputs) < 64:
            continue
        optimizer.zero_grad() #Set all graidents to zero for each step as they accumulate over backprop

        # forward + backward + optimize
        inputs = inputs.view(inputs.shape[0], -1)
        _,inputs,inputs2 = torch.tensor_split(inputs,((0,7)),-1)
        outputs = model.forward(inputs,inputs2).view(inputs.shape[0])
        #print(outputs[:5])
        labels = labels.float()
        loss = criterion(outputs,labels)

        # Calculate gradient of matrix with requires_grad = True
        loss.backward() #computes dloss/dx for every parameter x which has requires_grad=True
        optimizer.step() # x += -lr * x.grad ie updates the weights of the parameters
        # Adding loss to total loss
        total_loss += loss.item()
    end_time = time.time() - start_time

    # Printing out statistics
    print("Epoch no.",epoch+1 , round(0, 3),"%", "|total_loss: ", total_loss, "| epoch_duration: ", round(end_time,2),"sec")
    epoch += 1
    print(outputs[:10])
    if abs(total_loss - tl0) < 1:
        break
    else:
        tl0 = total_loss
inputs = torch.tensor(test_x.values)
labels = torch.tensor(test_y.values)
# inputs = inputs.view(inputs.shape[0], -1)
outputs = model.forward(inputs,None)
outputs = outputs[0].view(inputs.shape[0])
metricR2 = metrics.R2Score()
metricR2.update(outputs,labels)
print("R2",metricR2.compute())

[[49. 58. 62. ...  0.  1.  0.]
 [57. 64. 46. ...  0.  1.  0.]
 [90. 91. 91. ...  0.  1.  0.]
 ...
 [87. 88. 87. ...  0.  0.  1.]
 [90. 87. 88. ...  0.  0.  1.]
 [57. 61. 76. ...  0.  0.  1.]]
Epoch no. 1 0 % |total_loss:  4613967.016494751 | epoch_duration:  14.7 sec
tensor([6.5231, 4.0229, 3.5935, 6.1075, 5.3612, 5.2792, 6.7285, 6.2363, 6.1386,
        6.6137], grad_fn=<SliceBackward0>)
Epoch no. 2 0 % |total_loss:  4277077.119384766 | epoch_duration:  9.75 sec
tensor([126.8399,  79.0681,  69.8445, 119.9180,  96.5752, 100.2279, 126.2088,
        121.4228, 117.6229, 124.4627], grad_fn=<SliceBackward0>)
Epoch no. 3 0 % |total_loss:  4291664.039161682 | epoch_duration:  21.07 sec
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], grad_fn=<SliceBackward0>)
Epoch no. 4 0 % |total_loss:  4260525.412567139 | epoch_duration:  10.35 sec
tensor([79.7684, 50.6585, 44.8584, 76.2479, 59.6183, 63.0219, 81.6568, 75.9692,
        75.4932, 78.3317], grad_fn=<SliceBackward0>)
Epoch no. 5 0 % |total_loss:

KeyboardInterrupt: 